<a href="https://colab.research.google.com/github/xedinseu/Yolov4_for_schemes/blob/main/%D0%9F%D0%BE%D0%B4%D0%B3%D0%BE%D0%BD%D0%BA%D0%B0_%D0%B8%D0%B7%D0%BE%D0%B1%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D0%B9_%D0%BF%D0%BE_%D1%80%D0%B0%D0%B7%D1%80%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D1%8E_%D0%B4%D0%BB%D1%8F_YOLO_%D0%B8_%D0%B4%D1%80%D1%83%D0%B3%D0%B0%D1%8F_%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image # Модули работы с изображениями
import os
import numpy as np
import matplotlib.pyplot as plt
import copy

In [ ]:
from google.colab import drive # подключаем гугл драйв диск для загрузки файлов
drive.mount('/content/drive')

Mounted at /content/drive


# Приведение изображений к нужному разрешению

In [ ]:
path = '/content/drive/MyDrive/Базы/text_cleaning/test/'
processed_path = '/content/test/'

In [ ]:
# Изменяем размер картинки под input_shape
files = os.listdir(path) # Получаем список файлов
k=0
for f in files:
  image = Image.open(path+f) # Загружаем изображение
  iw, ih = image.size
  w, h = (540, 420)
  scale = min(w / iw, h / ih)
  nw = int(iw * scale)
  nh = int(ih * scale)
  image_for_predict = image.resize((nw, nh), Image.BICUBIC)
  new_image = Image.new('L',(540,420), (128))
  new_image.paste(image_for_predict, ((w - nw) // 2, (h - nh) // 2))
  #new_image.convert('LA')
  new_image.save(processed_path+f.format(k))
  k+=1

In [ ]:
import shutil
# создаем архив с нашими текстовыми файлами для скачивания
shutil.make_archive("test", "zip", "test")

'/content/test.zip'

In [ ]:
# Изменяем размер картинки под input_shape не валидных
files = os.listdir(path) # Получаем список файлов
k=0
for f in files:
  image = Image.open(path+f) # Загружаем изображение
  iw, ih = image.size
  w, h = (1280, 1280)
  scale = min(w / iw, h / ih)
  nw = int(iw * scale)
  nh = int(ih * scale)
  image_for_predict = image.resize((nw, nh), Image.BICUBIC)
  new_image = Image.new('RGB', (1280,1280), (128, 128, 128))
  new_image.paste(image_for_predict, ((w - nw) // 2, (h - nh) // 2))
  #new_image.convert('LA')
  new_image.save(processed_path+'noyp_'+f[:-4]+'.png')
  k+=1

# Прогоняем изображение через вращение/масштабирование

In [ ]:
orig_path = '/content/temp/'
processed_path = '/content/data/'

In [ ]:
files = os.listdir(orig_path) # Получаем список файлов
k=0
for f in files:
  for i in range(3):
    # берем очередное изображение
    image = Image.open(orig_path+f) # Загружаем изображение
    # приводим его в нужный мастаб и ставим по центру серого поля
    new_scale = int(1280/(2**i))
    new_image = image.resize((new_scale, new_scale), Image.BICUBIC)
    image = Image.new('RGB', (1280,1280), (128, 128, 128))
    image.paste(new_image, (int(1280/2 - new_scale/2), int(1280/2 - new_scale/2)))
    image.save(processed_path+'{}_{}_1.png'.format(f[:-4],i,))
    # делаем 3 поворота по 90 градусов
    image = image.rotate(90)
    image.save(processed_path+'{}_{}_2.png'.format(f[:-4],i,))
    image = image.rotate(90)
    image.save(processed_path+'{}_{}_3.png'.format(f[:-4],i,))
    image = image.rotate(90)
    image.save(processed_path+'{}_{}_4.png'.format(f[:-4],i,))

# Прогоняем рамки через вращение/масштабирование

In [ ]:
orig_path = '/content/temp txt/'
processed_path = '/content/data/'

In [ ]:
files = os.listdir(orig_path) # Получаем список файлов
k=0
for file in files:
  f = open(orig_path+file, 'r')
  # считываем данные из файла в массив
  big_list = []
  for line in f:
    a = line
    my_list = []
    for i in range(5):
      place = a.find(' ')
      a_1 = a[:place]
      a = a[place+1:]
      my_list.append(float(a_1))
    big_list.append(my_list)

  # в цикле масштабируем массив, записывая в новуй
  for i in range(3):
    scale = 2**i
    pros_list = copy.deepcopy(big_list)
    for j in range(len(big_list)):
      # приближаем координаты центра рамок к центру при масштабировании
      pros_list[j][1] = (0.5 - 0.5/scale) + (1/scale)*big_list[j][1]
      pros_list[j][2] = (0.5 - 0.5/scale) + (1/scale)*big_list[j][2]
      # пропорционально масштабу уменьшаем размер рамок
      for k in range(3,5): 
        pros_list[j][k] = big_list[j][k]/scale
      # отмечу что нулевой элемент массива - класс, не меняется
    my_str = ''
    # перегоняем данные обратно в строку для записи в файл
    for k in range(len(pros_list)):
      for j in range(5):
        if j>0:
          my_str = my_str + str(pros_list[k][j]) + ' '
        else:
          # первая цифра в строке - класс, его записываем целочислено
          my_str = my_str + str(int(pros_list[k][j])) + ' '
      my_str = my_str + '\n'
    # сохраняем файл
    f = open(processed_path + file[:-4]+'_{}_1.txt'.format(i), 'w')
    f.write(my_str)
    f.close()
    # начинаем вращение в временном массиве

    # поворот на 90 градусов
    pros_list_1 = copy.deepcopy(pros_list)
    for j in range(len(pros_list_1)):
      pros_list_1[j][1] = 1-pros_list[j][2]
      pros_list_1[j][2] = pros_list[j][1]
      pros_list_1[j][3] = pros_list[j][4]
      pros_list_1[j][4] = pros_list[j][3]
    my_str = ''
    # перегоняем данные обратно в строку для записи в файл
    for k in range(len(pros_list_1)):
      for j in range(5):
        if j>0:
          my_str = my_str + str(pros_list_1[k][j]) + ' '
        else:
          # первая цифра в строке - класс, его записываем целочислено
          my_str = my_str + str(int(pros_list_1[k][j])) + ' '
      my_str = my_str + '\n'
    # сохраняем файл
    f = open(processed_path + file[:-4]+'_{}_4.txt'.format(i), 'w')
    f.write(my_str)
    f.close()

    # поворот на 180 градусов
    pros_list_1 = copy.deepcopy(pros_list)
    for j in range(len(pros_list_1)):
      pros_list_1[j][1] = 1-pros_list[j][1]
      pros_list_1[j][2] = 1-pros_list[j][2]
      pros_list_1[j][3] = pros_list[j][3]
      pros_list_1[j][4] = pros_list[j][4]
    my_str = ''
    # перегоняем данные обратно в строку для записи в файл
    for k in range(len(pros_list_1)):
      for j in range(5):
        if j>0:
          my_str = my_str + str(pros_list_1[k][j]) + ' '
        else:
          # первая цифра в строке - класс, его записываем целочислено
          my_str = my_str + str(int(pros_list_1[k][j])) + ' '
      my_str = my_str + '\n'
    # сохраняем файл
    f = open(processed_path + file[:-4]+'_{}_3.txt'.format(i), 'w')
    f.write(my_str)
    f.close()

    # поворот на 270 градусов
    pros_list_1 = copy.deepcopy(pros_list)
    for j in range(len(pros_list_1)):
      pros_list_1[j][1] = pros_list[j][2]
      pros_list_1[j][2] = 1-pros_list[j][1]
      pros_list_1[j][3] = pros_list[j][4]
      pros_list_1[j][4] = pros_list[j][3]
    my_str = ''
    # перегоняем данные обратно в строку для записи в файл
    for k in range(len(pros_list_1)):
      for j in range(5):
        if j>0:
          my_str = my_str + str(pros_list_1[k][j]) + ' '
        else:
          # первая цифра в строке - класс, его записываем целочислено
          my_str = my_str + str(int(pros_list_1[k][j])) + ' '
      my_str = my_str + '\n'
    # сохраняем файл
    f = open(processed_path + file[:-4]+'_{}_2.txt'.format(i), 'w')
    f.write(my_str)
    f.close()

# парсим выходные данные YOLO_v4 darknet, чтобы получить из них пригодные для отображения в ImgLabel, редактирования и дальнейшего обучения

In [ ]:
# Составляем словарь из всех классов и их номеров
f = open('classes.txt', 'r')
classes = {}
k=0
for line in f:
  classes[line[:-1]] = k
  k+=1
f.close()
classes

{'GND': 5,
 'capacitor': 2,
 'chip': 9,
 'connector': 12,
 'diode': 8,
 'empty link': 11,
 'fuse': 15,
 'incandescent lamp': 14,
 'inductor': 16,
 'key': 3,
 'net': 7,
 'net dot': 6,
 'out net': 4,
 'resistor': 0,
 'special components': 13,
 'transformer': 10,
 'transistor': 1}

In [ ]:
# функция удаляет все указанные значения из списка
def remove_values_from_list(the_list, val):
   return [value for value in the_list if value != val]

In [ ]:
# начинаем парсить файл результатов
f = open('result.txt', 'r')
k=0
# служебная информация прокрутиться при открытом файле trash
f1 = open('trash.txt', 'w')
for line in f:
  # как только находим название файла, открываем соответствующий ему текстовый файл на запись, закрыв предыдущий
  if line.find('/content/drive/')!= -1:
    f1.close()
    f1 = open('data/scheme_{}.txt'.format(k), 'w')
    k+=1
  # начинаем построчно парсить результаты, записывая их построчно в файл для очередной схемы
  one_obj_list = ''
  string = line.split(' ') # преврящаем очередную строку в список
  # удаляем из списка все пробелы и пустые значения, чтобы очтались только цифры и слова
  string = remove_values_from_list(string, ' ')
  string = remove_values_from_list(string, '')
  if string[0][:-1] in classes.keys(): # если первое значение списка есть в нашем словаре
  # то вычленяем из списка 4 значения для баундинг бокса и приводим их в нужный формат
    one_obj_list = one_obj_list + str(classes[string[0][:-1]]) + ' '
    one_obj_list = one_obj_list + str(int(string[2])/1280 + int(string[6])/1280/2) + ' '
    one_obj_list = one_obj_list + str(int(string[4])/1280 + int(string[8][:-2])/1280/2) + ' '
    one_obj_list = one_obj_list + str(int(string[6])/1280) + ' '
    one_obj_list = one_obj_list + str(int(string[8][:-2])/1280) + '\n'
    f1.write(one_obj_list)

# Уменьшаем подаваемые сети изображения

In [ ]:
orig_path = '/content/temp/'
processed_path = '/content/data/'

In [ ]:
files = os.listdir(orig_path) # Получаем список файлов
k=0
for f in files:
    # берем очередное изображение
    image = Image.open(orig_path+f) # Загружаем изображение
    # приводим его в нужный мастаб и ставим по центру серого поля
    new_scale = int(1280/2)
    new_image = image.resize((new_scale, new_scale), Image.BICUBIC)
    image = Image.new('RGB', (1280,1280), (128, 128, 128))
    image.paste(new_image, (int(1280/2 - new_scale/2), int(1280/2 - new_scale/2)))
    image.save(processed_path+f)

#изменяем классы

In [ ]:
orig_path = '/content/temp/temp/'
processed_path = '/content/data/'

In [ ]:
!unzip /content/temp.zip -d /content/temp/

Archive:  /content/temp.zip
   creating: /content/temp/temp/
  inflating: /content/temp/temp/0.jpg  
  inflating: /content/temp/temp/1.jpg  
  inflating: /content/temp/temp/10.jpg  
  inflating: /content/temp/temp/100.jpg  
 extracting: /content/temp/temp/1000.jpg  
  inflating: /content/temp/temp/1001.jpg  
  inflating: /content/temp/temp/1002.jpg  
  inflating: /content/temp/temp/1003.jpg  
  inflating: /content/temp/temp/1005.jpg  
  inflating: /content/temp/temp/1006.jpg  
  inflating: /content/temp/temp/1007.jpg  
  inflating: /content/temp/temp/1008.jpg  
  inflating: /content/temp/temp/1009.jpg  
  inflating: /content/temp/temp/101.jpg  
  inflating: /content/temp/temp/1010.jpg  
  inflating: /content/temp/temp/1011.jpg  
  inflating: /content/temp/temp/1012.jpg  
  inflating: /content/temp/temp/1014.jpg  
  inflating: /content/temp/temp/1015.jpg  
  inflating: /content/temp/temp/1016.jpg  
  inflating: /content/temp/temp/1017.jpg  
  inflating: /content/temp/temp/1018.jpg  
  i

4 класса

In [ ]:
import os
files = os.listdir(orig_path) # Получаем список файлов
for file in files:
  f = open(orig_path+file, 'r')
  # считываем данные из файла в массив
  big_list = []
  for line in f:
    if line[0] == '7':
      line='0' + line[1:]
    elif line[0] == '6':
      line='1' + line[1:]
    elif line[0] == '4':
      line='2' + line[1:]
    else:
      if line[1]!=' ':
        line='3' + line[2:]
      else:
        line='3' + line[1:]
    big_list.append(line)
  f.close()
  f = open(processed_path+file, 'w')
  for i in range (len(big_list)):
    f.write(big_list[i])
  f.close()

Компоненты

In [ ]:
import os
files = os.listdir(orig_path) # Получаем список файлов
for file in files:
  f = open(orig_path+file, 'r')
  # считываем данные из файла в массив
  big_list = []
  for line in f:
    if line[0] == '7':
      line=''
    elif line[0] == '6':
      line=''
    elif line[0] == '4':
      line=''
    elif line[0]=='5':
        line='4' + line[1:]
    elif line[0] == '9':
        line='5' + line[1:]
    else:
      if (line[0] == '0' or line[0] == '1' or line[0] == '2' or line[0] == '3') and line[1] == ' ':
        line = line
      elif line[1]!=' ':
        line=str(10+int(line[1])-3) + line[2:]
      else:
        line=str(int(line[0])-3) + line[1:]
    big_list.append(line)
  f.close()
  f = open(processed_path+file, 'w')
  for i in range (len(big_list)):
    f.write(big_list[i])
  f.close()

# Создание пустых txt файлов для невалидных картинок

In [ ]:
orig_path = '/content/temp/'
processed_path = '/content/data/'

In [ ]:
import os
files = os.listdir(orig_path) # Получаем список файлов
for file in files:
  f = open(processed_path+file[:-4]+'.txt', 'w')
  f.write('')
  f.close()

# Выкачиваем результаты из ноутбука

In [ ]:
import shutil
# создаем архив с нашими текстовыми файлами для скачивания
shutil.make_archive("data", "zip", "data")

'/content/data.zip'